In [1]:
RFTrackPath = '/home/pkruyt/cernbox/rf-track-2.0'
import sys
import numpy as np
sys.path.append(RFTrackPath)

import RF_Track as RFT





RF-Track, version 2.0.4

Copyright (C) 2022 CERN, Geneva, Switzerland. All rights not expressly
granted are reserved. See the COPYRIGHT file at the top-level directory
of the source code distribution.

Author and contact:
 Andrea Latina <andrea.latina@cern.ch>
 BE-ABP Group
 CERN
 CH-1211 GENEVA 23
 SWITZERLAND

RF-Track was compiled with GSL-2.5 and fftw-3.3.5-sse2-avx

This is free software; see the source code for copying conditions.
There is ABSOLUTELY NO WARRANTY; not even for MERCHANTABILITY or
FITNESS FOR A PARTICULAR PURPOSE.



In [2]:
# copyright ############################### #
# This file is part of the Xtrack Package.  #
# Copyright (c) CERN, 2021.                 #
# ######################################### #


import xtrack as xt
import xobjects as xo
import xpart as xp

import ducktrack as dtk
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
from tqdm import tqdm






beta_rel=0.09405801936973575
gamma = 1.004453024014182

current=0.6
length = 2.5 # m cooler length
r_beam=25*1e-3



T_perp = 0.1 # <E> [eV] = kb*T
T_l =  0.01 # <E> [eV]
magnetic_field = 0.075 # T for LEIR
B_ratio=1e-10


mass0=193729.0248722061*1e6 #eV/c2
c=299792458.0
p0c = mass0*beta_rel*gamma #eV/c

length = 78.54370266 #m
T = length/(c*beta_rel)
ms_per_turn = 1000*T

alpha_x=0
alpha_y=0
Q_x = 1.82; #% 2pi, tune x
Q_y = 2.72; #% 2pi, tune y
DQx=0
DQy=0
beta_x=5
beta_y=5
emittance=14*1e-6

disp_x=0


arc=xt.LinearTransferMatrix(Q_x=Q_x, Q_y=Q_y,
                            beta_x_0=beta_x, beta_x_1=beta_x,
                            beta_y_0=beta_y,  beta_y_1=beta_y,
                            alpha_x_0=0,   alpha_x_1=0,
                            alpha_y_0=0,   alpha_y_1=0,
                            disp_x_0=disp_x,disp_x_1=disp_x,
                            disp_y_0=0,    disp_y_1=0,
                            beta_s=1*1e40,
                            Q_s=0,
                            chroma_x=0.0, chroma_y=0
                                           
                 )



In [3]:
twiss = np.array([length,beta_x, alpha_x, Q_x, beta_y, alpha_y, Q_y])
transfer_line = RFT.TransferLine(twiss)


In [4]:
import RF_Track as RFT

Cooler_Vele=beta_rel
ne=8.343768e+13 # e/m^3 #3.945e+13
Cooler_Ne=ne
Neutralisation=True
radius_e_beam=r_beam

Cooler_Vele = 9.43015738e-02 #% c
Cooler_L = 1.5;# % m, cooler length
Cooler_B = 0.06; #% T 0.07
Cooler_r0 = 24.8;# % mm, electron beam radius
#Cooler_Ne = 8.27e13;# % electron number density I=0.6A
Cooler_Ne = 3.945e+13# 6.8e+13;# % electron number density #/m^3   I= 350mA
I=0.35
Cooler_beta = 1.9; #% m, beta function in the middle of the cooler
Cooler_beta_y = 6.4; #% m, beta function in the middle of the cooler
Cooler_Tr = 0.1; #% eV
Cooler_Tl = 0.01;# % eV
Neutralisation=0 # 0/1 -- 0==OFF, 1==ON
Dx=0 #dispersion m
Cooler_Gamma=(1/(Cooler_beta*1e3))


import sys
sys.path.append(RFTrackPath)
import RF_Track as RFT

Nx = 20
Ny = 20
Nz = 2

E = RFT.ElectronCooler(Cooler_L, Cooler_r0/1e3, Cooler_r0/1e3,ne,Cooler_Vele)
# E = RFT.ElectronCooler(Cooler_L, Cooler_r0/1e3, Cooler_r0/1e3)
E.set_Q(-1); #% electron charge
E.set_static_Bfield(0.0, 0.0, Cooler_B)
E.set_temperature(Cooler_Tr, Cooler_Tl)
E.set_electron_mesh(Nx, Ny, Nz, Cooler_Ne, 0, 0, Cooler_Vele)

In [5]:
num_particles=1000
#old
emittance=35*1e-6 #inital emittance
emittance=5*1e-6 #inital emittance

sigma_x = np.sqrt(beta_x*emittance)
sigma_px = np.sqrt(emittance*1/beta_x)
sigma_y = np.sqrt(beta_y*emittance)
sigma_py = np.sqrt(emittance*1/beta_y)
sigma_p = 5e-4

delta = np.random.normal(loc=0.0, scale=sigma_p,  size=num_particles)
x =     np.random.normal(loc=0.0, scale=sigma_x,  size=num_particles)+disp_x*delta
px =    np.random.normal(loc=0.0, scale=sigma_px, size=num_particles)
y =     np.random.normal(loc=0.0, scale=sigma_y,  size=num_particles)
py =    np.random.normal(loc=0.0, scale=sigma_py, size=num_particles)

t= np.random.normal(loc=0.0, scale=0.0, size=num_particles)
p = delta*p0c+p0c

particles = xp.Particles(
        
        mass0=mass0,
        q0=54,
        p0c=p0c,
        x=x,
        px=px,
        y=y,
        py=py,
        delta=delta,
        zeta=0)

cooler = xt.ElectronCooler(current=current,length=length,r_beam=r_beam,
                                         T_perp=T_perp,T_l=T_l,
                                         magnetic_field=magnetic_field,B_ratio=B_ratio,
                                         Neutralisation_space_charge=0)


# E = RFT.ElectronCooler(length, r_beam)
# E.set_Q(-1); #% electron charge
# E.set_static_Bfield(0.0, 0.0, magnetic_field)
# E.set_temperature(T_perp, T_l)

 


###########################################################################################################
from Xsuite_to_RF import XSUITE_TO_RF_converter
from RF_to_Xsuite import RF_TO_XSUITE_converter

beam=XSUITE_TO_RF_converter(particles)
# beam.set_coasting(length)


#B = RFT.Bunch6d(mass0, num_particles, 1, [x,px,y,py,t, p] )


num_turns = int(1e5)
cooler_boolean = True
data_num_turns=int(1e5)
save_interval = 100

lower_percentile = 2.5
upper_percentile = 97.5

In [6]:
# Create empty lists to store RF track data
x_rf_list = []
px_rf_list = []
hist_1_rf_list = []
hist_2_rf_list = []
width_rf_list = []

# Specify the file paths where you want to save the data
file_path_no_cooler_rf = 'results/rf_no_cooler.npz'
file_path_cooler_rf = 'results/rf_cooler.npz'

# Perform the loop
for i in tqdm(range(num_turns)):
    transfer_line.track(beam)

    if cooler_boolean:
        E.track(beam)

    if i % save_interval == 0:
        x_rf = beam.get_phase_space("%x") * 1e-3
        # print(x_rf)
        px_rf = beam.get_phase_space("%xp") * 1e-3
        x_rf_list.append(x_rf)
        px_rf_list.append(px_rf)
        hist_1_rf_list.append(np.percentile(x_rf, lower_percentile))
        hist_2_rf_list.append(np.percentile(x_rf, upper_percentile))
        width_rf_list.append(np.percentile(x_rf, upper_percentile) - np.percentile(x_rf, lower_percentile))

if num_turns >= data_num_turns:
    # Save the lists as a single compressed archive file
    if not cooler_boolean:
        np.savez(file_path_no_cooler_rf, x_rf=x_rf_list, px_rf=px_rf_list, hist_1_rf=hist_1_rf_list, hist_2_rf=hist_2_rf_list, width_rf=width_rf_list)
    if cooler_boolean:
        np.savez(file_path_cooler_rf, x_rf=x_rf_list, px_rf=px_rf_list, hist_1_rf=hist_1_rf_list, hist_2_rf=hist_2_rf_list, width_rf=width_rf_list)



In [ ]:
# Create empty lists to store data
x_xs_list = []
px_xs_list = []
hist_1_xs_list = []
hist_2_xs_list = []
width_xs_list = []

# Specify the file paths where you want to save the data
file_path_no_cooler = 'results/xsuite_no_cooler.npz'
file_path_cooler = 'results/xsuite_cooler.npz'

# Perform the loop
for i in tqdm(range(num_turns)):
    
    arc.track(particles)

    if cooler_boolean:
        cooler.track(particles)

    if i % save_interval == 0:
        x_xs = particles.x.copy()
        px_xs = particles.px.copy()
        x_xs_list.append(x_xs)
        px_xs_list.append(px_xs)
        hist_1_xs_list.append(np.percentile(x_xs, lower_percentile))
        hist_2_xs_list.append(np.percentile(x_xs, upper_percentile))
        width_xs_list.append(np.percentile(x_xs, upper_percentile) - np.percentile(x_xs, lower_percentile))

if num_turns >= data_num_turns:
    # Save the lists as a single compressed archive file
    if not cooler_boolean:
        np.savez(file_path_no_cooler, x_xs=x_xs_list, px_xs=px_xs_list, hist_1_xs=hist_1_xs_list, hist_2_xs=hist_2_xs_list, width_xs=width_xs_list)
    if cooler_boolean:
        np.savez(file_path_cooler, x_xs=x_xs_list, px_xs=px_xs_list, hist_1_xs=hist_1_xs_list, hist_2_xs=hist_2_xs_list, width_xs=width_xs_list)

